In [1]:
# from google.colab import drive
# drive.mount("/content/drive")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os
import math

# Load data

In [3]:
%pwd

'd:\\UIT\\Nam3_ky2\\ML\\CS114\\Final_project\\code'

In [4]:
# path = "/content/drive/MyDrive/ML/Final_Project/1. data_raw/"
# data = pd.DataFrame()
# for file in os.listdir(path):
#     df = pd.read_excel(path+file)
#     data = pd.concat([data, df])

df1 = pd.read_excel("../dataset/raw_data/negative_data.xlsx")
df2 = pd.read_excel("../dataset/raw_data/neutral_data.xlsx")
df3 = pd.read_excel("../dataset/raw_data/positive_data.xlsx")
data = pd.DataFrame()
data = pd.concat([data, df1, df2, df3])


In [5]:
data

,Rate,Review,Label
0,4.0,Mình thề là mình ko thể cảm nổi đồ ăn ở aeon m...,-1
1,3.8,Đôi khi thèm lên là bất chấp nắng nóng phi Và ...,-1
2,3.8,Ngõ treo biển cafe trứng đúng kiểu phố cổ hà n...,-1
3,3.8,Mình thấy địa chỉ cafe Giảng ở Nguyễn Hữu Huân...,-1
4,2.2,Mình là người Hà Nội và cũng cực kỳ khó tính t...,-1
...,...,...,...
6904,9.0,"Mình là khách hàng thường xuyên của quán, có t...",1
6905,7.2,Mình mới đến quán 1 lần và cảm thấy quán khá d...,1
6906,8.0,Quán ăn đồ Hàn mới mở ở gần trường mình nên bữ...,1
6907,10,Trà Sữa Dưa Lưới y như gu mình thích :))) Tuy ...,1


# Tiền xử lý

In [6]:
!pip install underthesea

In [7]:
import regex as re
import underthesea

## 1. Xoá HTML tag

In [8]:
def remove_html(text):
  return re.sub(r'<[^>]*>', '', text)

print(remove_html("<h1>Title</h1>"))

Title


## 2. Đánh dấu spam URL

In [9]:
def check_url(text):
    pattern = re.compile(r'(http|https)://[^\s]+')
    match = pattern.search(text)
    if match:
        return True
    else:
        return False

In [10]:
data.drop(data[data['Review'].apply(check_url)].index, inplace=True)
data.shape

(11481, 3)

## 3 Chuẩn hoá Unicode và dấu câu

In [11]:
from underthesea import text_normalize

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

# Hàm tạo dict để convert định dạng cũ sang định dạng mới
def loaddicchar():
  dic = {}
  char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split('|')
  charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split('|')
  for i in range(len(char1252)):
      dic[char1252[i]] = charutf8[i]
  return dic

dicchar = loaddicchar()

# Chuẩn hóa Unicode và dấu câu
def convert_unicode(txt):
    txt = re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)
    return text_normalize(txt)

print(convert_unicode("Ðảm baỏ chất lựơng phòng thí nghịêm hoá học"))

Đảm bảo chất lượng phòng thí nghiệm hóa học


## 4. Đưa về dạng viết thường

In [12]:
def to_lower_case(sentence):
    sentence = sentence.lower()
    return sentence

print(to_lower_case('Tôi là dân IT!'))

tôi là dân it!


## 5. Xoá các ký tự không cần thiết

In [13]:
def remove_unnecessary_charactor(document):
    # xóa các ký tự đặc biệt, emoji
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa kí tự chứa số
    document = re.sub(r'\w*\d\w*', '', document).strip()
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document

document = 'Biết 1505 là gì không? 😆'
remove_unnecessary_charactor(document)

'Biết là gì không'

## 6. Chuẩn hoá những từ lặp âm tiết

In [14]:
def remove_duplicate_characters(text):
    pattern = re.compile(r'(\w)\1{2,}')
    text = pattern.sub(r'\1', text)
    return text

print(remove_duplicate_characters("Trà sữa ngon nhân viên bán hàng nhiệt tình! Likeeeeeee!"))

Trà sữa ngon nhân viên bán hàng nhiệt tình! Like!


## 7. Chuẩn hoá viết tắt

In [15]:
import re
# Đọc các cặp giá trị từ file văn bản
replacements = {}
with open('abbreviations.txt', 'r') as file:
    for line in file:
        key, value = re.split(':', line, maxsplit=1)
        replacements[key] = value.strip()

def abbreviate(text):
    # Thay thế các giá trị trong chuỗi
    for key, value in replacements.items():
        text = re.sub(r'\b{}\b'.format(key), value, text)
    return(text)


In [16]:
text = "tôi hong iu k thick cá"
print(abbreviate(text))

tôi không yêu không thích cá


## Tổng hợp 1

In [17]:
def text_preprocessing1(document):
  document = remove_html(document)
  document = convert_unicode(document)
  document = to_lower_case(document)
  document = remove_unnecessary_charactor(document)
  document = remove_duplicate_characters(document)
  document = re.sub(r'\s+', ' ', document).strip() # xóa khoảng trắng thừa
  document = abbreviate(document)
  return document

text_preprocessing1('theo nghị định đã ban hànhhhhh, người dân cần ở trong nhà đến hết ngày 16/9.')

'theo nghị định đã ban hành người dân cần ở trong nhà đến hết ngày'

In [18]:
data["Preprocess_Review"] = data["Review"].apply(lambda x: text_preprocessing1(x))
data

,Rate,Review,Label,Preprocess_Review
0,4.0,Mình thề là mình ko thể cảm nổi đồ ăn ở aeon m...,-1,mình thề là mình không thể cảm nổi đồ ăn ở aeo...
1,3.8,Đôi khi thèm lên là bất chấp nắng nóng phi Và ...,-1,đôi khi thèm lên là bất chấp nắng nóng phi và ...
2,3.8,Ngõ treo biển cafe trứng đúng kiểu phố cổ hà n...,-1,ngõ treo biển cafe trứng đúng kiểu phố cổ hà n...
3,3.8,Mình thấy địa chỉ cafe Giảng ở Nguyễn Hữu Huân...,-1,mình thấy địa chỉ cafe giảng ở nguyễn hữu huân...
4,2.2,Mình là người Hà Nội và cũng cực kỳ khó tính t...,-1,mình là người hà nội và cũng cực kỳ khó tính t...
...,...,...,...,...
6904,9.0,"Mình là khách hàng thường xuyên của quán, có t...",1,mình là khách hàng thường xuyên của quán có tu...
6905,7.2,Mình mới đến quán 1 lần và cảm thấy quán khá d...,1,mình mới đến quán lần và cảm thấy quán khá dễ ...
6906,8.0,Quán ăn đồ Hàn mới mở ở gần trường mình nên bữ...,1,quán ăn đồ hàn mới mở ở gần trường mình nên bữ...
6907,10,Trà Sữa Dưa Lưới y như gu mình thích :))) Tuy ...,1,trà sữa dưa lưới y như gu mình thích tuy vẫn c...


## 8. Tách từ tiếng Việt

In [19]:
from underthesea import word_tokenize

sentence = 'Xin chào, rất vui được làm quen, tôi đến từ đất nước Việt Nam!'
print(word_tokenize(sentence))
print(word_tokenize(sentence, format='text'))

['Xin', 'chào', ',', 'rất', 'vui', 'được', 'làm quen', ',', 'tôi', 'đến', 'từ', 'đất nước', 'Việt Nam', '!']
Xin chào , rất vui được làm_quen , tôi đến từ đất_nước Việt_Nam !


## 9. Loại bỏ stopword

Stopword là những từ thường xuyên xuất hiện trong văn nói, văn viết và không có quá nhiều ý nghĩa, đặc trưng cho bài toán phân loại cảm xúc trong câu. Ví dụ về stopword như: là, rằng, theo, thì, đó,...

### 9.1. Xử lý với stopwords đã tổng hợp

In [20]:
%pwd

'd:\\UIT\\Nam3_ky2\\ML\\CS114\\Final_project\\code'

In [21]:
stopwords = open('stopword.txt')
stopwords = stopwords.readlines()
stopwords = [x.strip() for x in stopwords]

In [22]:
len(stopwords)

1814

In [23]:
def remove_stopwords(document):
  words = document.split(' ')
  res = list()
  for word in words:
    if word not in stopwords:
      res.append(word)

  return ' '.join(res)

print(text_preprocessing1('Nhân viên hơi thái_độ Giá cũng cao hơn các chỗ khác'))
print(remove_stopwords(text_preprocessing1('Nhân viên hơi thái_độ Giá cũng cao hơn các chỗ')))

nhân viên hơi thái_độ giá cũng cao hơn các chỗ khác
nhân viên hơi thái_độ giá cao hơn chỗ


# Tổng hợp 2

In [24]:
def text_preprocessing2(document):
  document = word_tokenize(document, format='text')
  document = remove_stopwords(document)
  return document

In [25]:
data["Preprocess_Review"] = data["Preprocess_Review"].apply(lambda x: text_preprocessing2(x))
data

,Rate,Review,Label,Preprocess_Review
0,4.0,Mình thề là mình ko thể cảm nổi đồ ăn ở aeon m...,-1,thề không_thể cảm nổi đồ ăn aeon mall ăn food ...
1,3.8,Đôi khi thèm lên là bất chấp nắng nóng phi Và ...,-1,đôi thèm bất_chấp nắng nóng phi đóng_đô quầy f...
2,3.8,Ngõ treo biển cafe trứng đúng kiểu phố cổ hà n...,-1,ngõ treo biển cafe trứng phố cổ hà_nội sâu nhỏ...
3,3.8,Mình thấy địa chỉ cafe Giảng ở Nguyễn Hữu Huân...,-1,địa_chỉ cafe giảng nguyễn_hữu_huân ngon hồi cò...
4,2.2,Mình là người Hà Nội và cũng cực kỳ khó tính t...,-1,hà_nội cực_kỳ khó_tính ăn phố gần cafe cũ giản...
...,...,...,...,...
6904,9.0,"Mình là khách hàng thường xuyên của quán, có t...",1,khách_hàng thường_xuyên tuần ăn thích nhất súp...
6905,7.2,Mình mới đến quán 1 lần và cảm thấy quán khá d...,1,khá dễ tìm thân_thiện đồ ăn_theo phong_cách hà...
6906,8.0,Quán ăn đồ Hàn mới mở ở gần trường mình nên bữ...,1,ăn đồ hàn gần trường nên bữa thèm nên quất nằm...
6907,10,Trà Sữa Dưa Lưới y như gu mình thích :))) Tuy ...,1,dưa lưới y_như gu thích tuy vẫn dưa lưới nhưng...


## 10. Bỏ các giá trị NaN, giá trị rỗng, lặp lại

In [26]:
# data.drop_duplicates(inplace=True)
# data

In [27]:
data.isna().sum()

Rate                 0
Review               0
Label                0
Preprocess_Review    0
dtype: int64

## Lưu dữ liệu đã tiền xử lý

In [28]:
data.to_excel("../dataset/final_data.1.xlsx", index=False)
